# Data Preprocessing for predicting Transcription Rate (TS)

This notebook is ment to convert raw cell data from several wells into multichannel images (along with its corresponding metadata).

Data was taken from:
`/storage/groups/ml01/datasets/raw/20201020_Pelkmans_NascentRNA_hannah.spitzer/` and server `vicb-submit-01`. 

Load libraries:

In [ ]:
import numpy as np
import pandas as pd
# To display all the columns
pd.options.display.max_columns = None
import os
import sys
import matplotlib.pyplot as plt
import json
import math

Load Parameters:

In [ ]:
# Do not touch the value of PARAMETERS_FILE!
# When this notebook is executed with jupyter-nbconvert (from script), 
# it will be replaced outomatically
PARAMETERS_FILE = 'dont_touch_me-input_parameters_file'
if not os.path.exists(PARAMETERS_FILE):
    raise Exception('Parameter file {} does not exist!'.format(PARAMETERS_FILE))
    
# Open parameters
with open(PARAMETERS_FILE) as params_file:
    p = json.load(params_file)
p.keys()

Take a look into the loaded parameters:

In [ ]:
p

Set paths and Load external libraries:

In [ ]:
# Load data path
DATA_DIR = p['raw_data_dir']
if not os.path.exists(DATA_DIR):
    raise Exception('Data path {} does not exist!'.format(DATA_DIR))
else:
    print('DATA_DIR: {}'.format(DATA_DIR))

# Load external libraries path
EXTERNAL_LIBS_PATH = p['external_libs_path']
if not os.path.exists(EXTERNAL_LIBS_PATH):
    raise Exception('External library path {} does not exist!'.format(EXTERNAL_LIBS_PATH))
else:
    print('EXTERNAL_LIBS_PATH: {}'.format(EXTERNAL_LIBS_PATH))
# Add EXTERNAL_LIBS_PATH to sys paths (for loading libraries)
sys.path.insert(1, EXTERNAL_LIBS_PATH)
# Load external libraries
from pelkmans.mpp_data import MPPData as MPPData
from pelkmans.mpp_data import normalize_and_save_MPPData_images as normalize_and_save
from pelkmans.mpp_data import get_image_normalization_vals as get_normalization_vals
from pelkmans.mpp_data import get_concatenated_metadata as get_concatenated_metadata

# Set logging configuration
import logging
logging.basicConfig(
    filename=p['log_file'],
    filemode='w', 
    level=getattr(logging, p['log_level'])
)
logging.info('Parameters loaded from file:\n{}'.format(PARAMETERS_FILE))

Check available data (Perturbations and Wells):

In [ ]:
logging.info('Reading local available perturbations-wells...')
# Save available local Perturbations and Wells
perturbations = [per for per in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, per))]
local_data = {}
#print('Local available perturbations-wells:\n')
for per in perturbations:
    pertur_dir = os.path.join(DATA_DIR, per)
    wells = [w for w in os.listdir(pertur_dir) if os.path.isdir(os.path.join(pertur_dir, w))]
    #print('{}\n\t{}\n'.format(p, wells))
    local_data[per] = wells

Select Perturbations and its wells to process: 

In [ ]:
msg = 'Local available perturbations-wells:\n{}'.format(local_data)
print(msg)
logging.debug(msg)

# In case you only want to load some specific perturbations and/or wells here:
#selected_data = {
#    '184A1_hannah_unperturbed': ['I11', 'I09'],
#    '184A1_hannah_TSA': ['J20', 'I16'],
#}

# Load perturbations-wells from parameters file
selected_data = p['perturbations_and_wells']

print('\nSelected perturbations-wells:\n{}'.format(selected_data))

#Generate and save data dirs
data_dirs = []
for per in selected_data.keys():
    for w in selected_data[per]:
        d = os.path.join(DATA_DIR, per, w)
        data_dirs.append(d)
        if not os.path.exists(d):
            msg = '{} does not exist!\nCheck if selected_data contain elements only from local_data dict.'.format(d)
            logging.error(msg)
            raise Exception(msg)
p['data_dirs'] = data_dirs

Process data:

In [ ]:
logging.info('Starting data loading process...')

data = {
    'train':[],
    'val':[],
    'test':[]
}

for data_dir in p['data_dirs']:
    msg = 'Processing dir {}...'.format(data_dir)
    logging.info(msg)
    print('\n\n'+msg)
    # Load data as an MPPData object
    mpp_temp = MPPData.from_data_dir(data_dir,
                                     dir_type=p['dir_type'],
                                     seed=p['seed'])
    
    # Add cell cycle to metadata (G1, S, G2)
    # Important! If mapobject_id_cell is not in cell_cycle_file =>
    # its corresponding cell is in Mitosis phase!
    if p['add_cell_cycle_to_metadata']:
        msg = 'Adding cell cycle to metadata...'
        logging.info(msg)
        print(msg)
        mpp_temp.add_cell_cycle_to_metadata(os.path.join(DATA_DIR, p['cell_cycle_file']))
    
    # Add well info to metadata
    if p['add_well_info_to_metadata']:
        msg = 'Adding well info to metadata...'
        logging.info('Adding well info to metadata...')
        print(msg)
        mpp_temp.add_well_info_to_metadata(os.path.join(DATA_DIR, p['well_info_file']))
    
    # Remove unwanted cells
    if p.get('filter_criteria', None) is not None:
        msg = 'Removing unwanted cells...'
        logging.info(msg)
        print(msg)
        mpp_temp.filter_cells(p['filter_criteria'], p['filter_values'])

    # Subtract background  values for each channel
    if p['subtract_background']:
        print('Subtracting background...')
        mpp_temp.subtract_background(os.path.join(DATA_DIR, p['background_value']))
    
    # Project every uni-channel images into a scalar for further analysis
    if p['project_into_scalar']:
        msg = 'Projecting data...'
        logging.info(msg)
        print(msg)
        mpp_temp.add_scalar_projection(p['method'])
        
    # Split data into train, validation and test
    msg = 'Spliting data into train, validation and test'
    logging.info(msg)
    print(msg)
    train_temp, val_temp, test_temp = mpp_temp.train_val_test_split(p['train_frac'], p['val_frac'])
    del(mpp_temp)
    
    if p['convert_into_image']:
        msg = 'Converting data into images...'
        logging.info(msg)
        print(msg)
        train_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
        msg = 'Train dataset converted'
        logging.info(msg)
        print(msg)
        val_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
        msg = 'Validation dataset converted'
        logging.info(msg)
        print(msg)
        test_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
        msg = 'Test dataset converted'
        logging.info(msg)
        print(msg)
    
    # Validate same channels across wells
    if len(data['val']) > 0:
        if not all(data['val'][0].channels.name == val_temp.channels.name):
            raise Exception('Channels across MPPData instances are not the same!')
            
    data['val'].append(val_temp)
    del(val_temp)
    data['test'].append(test_temp)
    del(test_temp)
    data['train'].append(train_temp)
    del(train_temp)
    
    

Normalize data:

In [ ]:
# Normalize train, val and test using the normalization parameters
# got from the train data (inner percentile% of train data)
if p['normalise']:
    msg = 'Normalizing data...'
    logging.info(msg)
    if p['convert_into_image']:
        # Onle get the normalization parameters here and normalize 
        # images during saving into files
        rescale_values = get_normalization_vals(
            instance_dict=data['train'], 
            percentile=p['percentile'])
    if not p['remove_original_data']:
        # Get normalization values and normalize original MPPData
        pass
        # TODO: Rewrite rescale_intensities_per_channel to deal with
        # a dictionary of MPPData instances, instead of a big numpy array
        # (this is to avoid the duplication in memory)
        #rescale_values = train.rescale_intensities_per_channel(percentile=p['percentile'], )
        #_ = val.rescale_intensities_per_channel(rescale_values=rescale_values)
        #_ = test.rescale_intensities_per_channel(rescale_values=rescale_values)
    p['normalise_rescale_values'] = list(rescale_values)

In [ ]:
# Merge metadata and Normalize projected values
msg = 'Normalizing Projected data...'
logging.info(msg)
    
metadata, rescale_values = get_concatenated_metadata(
    mppdata_dict=data,
    normalize=(p['normalise'] & p['project_into_scalar']),
    norm_key='train',
    projection_method=p['method'],
    percentile=p['percentile'])

p['normalise_rescale_values_scalars'] = list(rescale_values)

In [ ]:
metadata

## Save data

Prepare to save data:

In [ ]:
import shutil

msg = 'Starting data saving process...'
logging.info(msg)

# create dir
outdir = p['output_data_dir']
if os.path.exists(outdir):
    msg = 'Warning! Directory {} already exist! Deleting...\n'.format(outdir)
    logging.info(msg)
    print(msg)
    try:
        shutil.rmtree(outdir)
    except OSError as e:
        msg  = 'Dir {} could not be deleted!\n\nOSError: {}'.format(outdir, e)
        logging.info(msg)
        print(msg)

msg = 'Creating dir: {}'.format(outdir)
logging.info(msg)
print(msg)
os.makedirs(outdir, exist_ok=False)
    

In [ ]:
# Get channels ids (proteins) which will be used to predict transcripcion rate
input_ids = list(data['val'][0].channels.set_index('name').loc[p['input_channels']]['channel_id'])
# Get id of the channel that measure trancripcion rate
output_ids = list(data['val'][0].channels.set_index('name').loc[p['output_channels']]['channel_id'])
# add output channel id after the input channels ids
channels_ids = input_ids + output_ids

Save Images and masks

In [ ]:
# Note! instead of calculating the response value (y) here and save
# it separatelly, we will do it on the modeling part

if p['convert_into_image']: 
    msg = 'Saving images and masks...'
    logging.info(msg)
    
    output_files = normalize_and_save(
        mppdata_dict=data,
        norm_vals=np.array(p['normalise_rescale_values']),
        channels_ids=channels_ids,
        outdir=outdir
    )
    p['output_files'] = output_files
    print(output_files)
    

Save metadata and used parameters

In [ ]:
msg = 'Saving Parameters and Metadata...'
logging.info(msg)

# save params
json.dump(p, open(os.path.join(outdir, 'params.json'), 'w'), indent=4)

# save metadata
metadata.to_csv(os.path.join(outdir, 'metadata.csv'))

# Save used channels
#train.channels.to_csv(os.path.join(outdir, 'channels.csv'))
k=list(data.keys())[0]
data[k][0].channels.set_index('channel_id').loc[channels_ids].to_csv(os.path.join(outdir, 'channels.csv'))